In [ ]:
# 2.8 Encoding word posiotions

import torch
import torch.utils.data as data
import tiktoken

# Using dataset and laoder from 2.6
max_len = 4
# Creating dataset
class GPTDatasetV1(data.Dataset):
    def __init__(self, txt, tokenizer, max_len, stride = 1):
        super().__init__()
        self.input_ids = []
        self.target_ids = []

        # Getting indexes
        torken_ids = tokenizer.encode(txt)

        # Making shift as in previos example
        for i in range(0, len(torken_ids) - max_len, stride ):
            input_chunk = torken_ids[i: i+max_len]
            target_chunk = torken_ids[i+1 : i + max_len +1 ]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __getitem__(self, index):
        return self.input_ids[index] , self.target_ids[index]
    
    def __len__(self):
        return len(self.input_ids)

# creating Dataloader
tokenizer = tiktoken.get_encoding("gpt2")

with open ("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

dataset = GPTDatasetV1(raw_text, tokenizer, max_len=max_len, stride=4)
d_train = data.DataLoader(dataset,batch_size=8, shuffle=False)

data_iter = iter(d_train)
inputs, targets = next(data_iter)
print(f"Inputs \n {inputs} \nTargets \n {targets} \nInputs shape:\n {inputs.shape}")


Inputs 
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]]) 
Targets 
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]]) 
Inputs shape:
 torch.Size([8, 4])


In [ ]:
# For ex we take 256-dim vector of embedding size for BPE encoder dict
vocab_size = 50257
output_dim = 256
token_embeddin_layer = torch.nn.Embedding(vocab_size, output_dim)

# Embend this inputs
token_embeddings = token_embeddin_layer(inputs)
print(token_embeddings.shape)

# torch.Size([8, 4, 256])
# 8 -batches, 4 -tokens, 256 -embeddings

torch.Size([8, 4, 256])


In [4]:
# Creating positional embeddings
max_len = 4
context_len = max_len
pos_embedding_layer = torch.nn.Embedding(context_len, output_dim)
pos_embeddings = pos_embedding_layer(torch.arange(context_len))
print(pos_embeddings.shape)

torch.Size([4, 256])


In [5]:
# Creation of input embeddings
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])
